In [65]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [66]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

In [67]:
kospi_train = fdr.DataReader('KS11', '2015-01-01','2021-03-31')
kospi_train['Change1'] = kospi_train['Close']/kospi_train['Close'].shift(1) - 1

kospi_train = kospi_train.dropna()
print(kospi_train.head())
print(kospi_train.columns)
print(kospi_train.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2015-01-05  1921.959961  1921.959961  1903.369995  1915.750000  1915.750000   
2015-01-06  1895.479980  1900.020020  1877.380005  1882.449951  1882.449951   
2015-01-07  1878.900024  1887.599976  1876.270020  1883.829956  1883.829956   
2015-01-08  1895.849976  1907.229980  1891.770020  1904.650024  1904.650024   
2015-01-09  1919.800049  1929.099976  1915.849976  1924.699951  1924.699951   

              Volume   Change1  
Date                            
2015-01-05  313400.0 -0.005549  
2015-01-06  303700.0 -0.017382  
2015-01-07  280500.0  0.000733  
2015-01-08  262200.0  0.011052  
2015-01-09  308800.0  0.010527  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(1531, 7)


In [68]:
kospi_test = fdr.DataReader('KS11', '2021-04-01','2022-10-31')
kospi_test['Change1'] = kospi_test['Close']/kospi_test['Close'].shift(1) - 1

kospi_test = kospi_test.dropna()
print(kospi_test.head())
print(kospi_test.columns)
print(kospi_test.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2021-04-02  3104.719971  3121.580078  3101.600098  3112.800049  3112.800049   
2021-04-05  3121.229980  3127.199951  3101.860107  3120.830078  3120.830078   
2021-04-06  3123.800049  3139.290039  3110.189941  3127.080078  3127.080078   
2021-04-07  3129.070068  3146.189941  3125.330078  3137.409912  3137.409912   
2021-04-08  3137.250000  3145.810059  3122.090088  3143.260010  3143.260010   

             Volume   Change1  
Date                           
2021-04-02   792700  0.008227  
2021-04-05  1084800  0.002580  
2021-04-06  1279000  0.002003  
2021-04-07  1984800  0.003303  
2021-04-08  1357300  0.001865  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(388, 7)


In [69]:
kospi_train.index = kospi_train.index.strftime('%Y-%m-%d')
print(kospi_train.index)

kospi_test.index = kospi_test.index.strftime('%Y-%m-%d')
print(kospi_test.index)

Index(['2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09',
       '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16',
       ...
       '2021-03-18', '2021-03-19', '2021-03-22', '2021-03-23', '2021-03-24',
       '2021-03-25', '2021-03-26', '2021-03-29', '2021-03-30', '2021-03-31'],
      dtype='object', name='Date', length=1531)
Index(['2021-04-02', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
       '2021-04-09', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15',
       ...
       '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21', '2022-10-24',
       '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', '2022-10-31'],
      dtype='object', name='Date', length=388)


In [70]:
import json
embedding = open('/home/work/nlp/embedding/result/news_title_128.json', encoding='utf-8')
em_dict = json.load(embedding)

In [71]:
ke_vec_train = []
ke_vec_test = []

for index, row in kospi_train.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Close'])
    ke_vec_train.append(a)

for index, row in kospi_test.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Close'])
    ke_vec_test.append(a)

In [72]:
print(ke_vec_train[0], ke_vec_test[0])

[1915.75, 0.0033641590271145105, 0.22497211396694183, -0.14655791223049164, -0.37470588088035583, -0.13409985601902008, 0.13826999068260193, 0.0019116881303489208, 0.10584980994462967, 0.23014280200004578, -0.13657808303833008, 0.052695102989673615, -0.0011661400785669684, -0.11453178524971008, -0.09622907638549805, 0.17620177567005157, -0.3827759027481079, -0.14051778614521027, -0.30580857396125793, 0.32278838753700256, -0.10491470992565155, -0.2664456069469452, -0.3328095078468323, 0.09656387567520142, -0.08282316476106644, 0.3154531717300415, 0.013810266740620136, 0.1514519453048706, 0.12658272683620453, -0.030929693952202797, 0.24329601228237152, -0.2921745777130127, -0.0768013522028923, -0.009143466129899025, -0.03771670535206795, -0.11348088085651398, -0.39978331327438354, -0.1786743700504303, -0.12624947726726532, -0.17722634971141815, -0.25062593817710876, -0.2783448100090027, -0.13989445567131042, 0.10892444849014282, -0.1342029720544815, 0.5296337008476257, 0.2970207631587982

In [73]:
kospi_train_c1 = kospi_train['Change1']
kospi_train_c1 = 100 * kospi_train_c1.values

kospi_test_c1 = kospi_test['Change1']
kospi_test_c1 = 100 * kospi_test_c1.values

In [74]:
lookback_window = 60
x_train_1, y_train_1 = [], []
x_test_1, y_test_1 = [], []

In [75]:
for i in range(lookback_window, len(ke_vec_train)-1):
    x_train_1.append(ke_vec_train[i - lookback_window:i])
    y_train_1.append(kospi_train_c1[i])
    
for index in range(len(y_train_1)):
    if y_train_1[index] >= 0:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease' 

In [76]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
print(x_train_1.shape, y_train_1.shape)

(1470, 60, 129) (1470,)


In [77]:
for i in range(lookback_window, len(ke_vec_test)-1):
    x_test_1.append(ke_vec_test[i - lookback_window:i])
    y_test_1.append(kospi_test_c1[i])
    
for index in range(len(y_test_1)):
    if y_test_1[index] >= 0:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'    

In [78]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
print(x_test_1.shape, y_test_1.shape)

(327, 60, 129) (327,)


In [79]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
y_test_1_e = to_categorical(encoder.transform(y_test_1))

print(y_train_1_e.shape, y_test_1_e.shape)

(1470, 2) (327, 2)


In [80]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=2,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128],
        use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_2 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=3,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128],
        use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_2.summary()
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_3 = Sequential([
    TCN(input_shape=(lookback_window, 129),
        nb_filters=100,
        kernel_size=4,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128],
        use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_30 (TCN)                (None, 100)               346800    
                                                                 
 dense_30 (Dense)            (None, 2)                 202       
                                                                 
Total params: 347,002
Trainable params: 343,802
Non-trainable params: 3,200
_________________________________________________________________
Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_31 (TCN)                (None, 100)               509700    
                                                                 
 dense_31 (Dense)            (None, 2)                 202       
                                                                 
Total params: 509,90

In [81]:
print('Train model_1...')
model_1.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

print('Train model_2...')
model_2.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

print('Train model_3...')
model_3.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

Train model_1...
Epoch 1/50
46/46 [==============================] - 15s 59ms/step - loss: 1.0251 - accuracy: 0.5082
Epoch 2/50
46/46 [==============================] - 2s 40ms/step - loss: 0.8272 - accuracy: 0.4980
Epoch 3/50
46/46 [==============================] - 2s 38ms/step - loss: 0.7651 - accuracy: 0.5361
Epoch 4/50
46/46 [==============================] - 2s 39ms/step - loss: 0.7217 - accuracy: 0.5531
Epoch 5/50
46/46 [==============================] - 2s 39ms/step - loss: 0.7707 - accuracy: 0.5381
Epoch 6/50
46/46 [==============================] - 2s 37ms/step - loss: 0.7328 - accuracy: 0.5429
Epoch 7/50
46/46 [==============================] - 2s 38ms/step - loss: 0.7063 - accuracy: 0.5741
Epoch 8/50
46/46 [==============================] - 2s 38ms/step - loss: 0.6920 - accuracy: 0.5776
Epoch 9/50
46/46 [==============================] - 2s 40ms/step - loss: 0.7223 - accuracy: 0.5646
Epoch 10/50
46/46 [==============================] - 2s 39ms/step - loss: 0.7012 - accuracy

46/46 [==============================] - 2s 37ms/step - loss: 0.5189 - accuracy: 0.7367
Epoch 33/50
46/46 [==============================] - 2s 37ms/step - loss: 0.5077 - accuracy: 0.7551
Epoch 34/50
46/46 [==============================] - 2s 39ms/step - loss: 0.5217 - accuracy: 0.7456
Epoch 35/50
46/46 [==============================] - 2s 39ms/step - loss: 0.4937 - accuracy: 0.7741
Epoch 36/50
46/46 [==============================] - 2s 38ms/step - loss: 0.4650 - accuracy: 0.7694
Epoch 37/50
46/46 [==============================] - 2s 38ms/step - loss: 0.5095 - accuracy: 0.7463
Epoch 38/50
46/46 [==============================] - 2s 37ms/step - loss: 0.5017 - accuracy: 0.7667
Epoch 39/50
46/46 [==============================] - 2s 39ms/step - loss: 0.4524 - accuracy: 0.7898
Epoch 40/50
46/46 [==============================] - 2s 37ms/step - loss: 0.4727 - accuracy: 0.7789
Epoch 41/50
46/46 [==============================] - 2s 38ms/step - loss: 0.4777 - accuracy: 0.7626
Epoch 42/50


In [82]:
model_1.evaluate(x_test_1, y_test_1_e)[1]
model_2.evaluate(x_test_1, y_test_1_e)[1]
model_3.evaluate(x_test_1, y_test_1_e)[1]

11/11 [==============================] - 1s 57ms/step - loss: 8.4629 - accuracy: 0.4832


0.483180433511734